In [ ]:
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pickle

df = pd.read_csv('/content/hate_speech.tsv', sep='\t').fillna(' ') #read_excel('/content/train.xlsx')
print(df.head())
# Create a list of punctuation marks
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '•', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

# Code to replace punctuations with whitespaces
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' ')
    return x
df['Cleaned'] = df['Text'].apply(lambda x: re.sub(r'http\S+', '', x))
df['Cleaned'] = df['Cleaned'].apply(lambda x: re.sub("@[\w]*", '', x))
df['Cleaned'] = df['Cleaned'].apply(lambda x: clean_text(x))
df['Cleaned'] = df['Cleaned'].str.lower()
df['Cleaned'] = df['Cleaned'].apply(lambda x:' '.join(x.split()))
df['Sentiment'] = df['Label']
df = df.drop(['Text','Label'],axis=1)
print(df.head())
batch=df[0:4000]
!pip install transformers
import torch
from torch.autograd import Variable
from torch import IntTensor
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")

tokenized = batch['Cleaned'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=40,truncation=True)))
max_len = 0
for i in tokenized.values:
  #print(len(i))
  if len(i) > max_len:
    max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

print(padded)
output = model(padded)
embedding=output.last_hidden_state
features = embedding[:,0,:].numpy()
y =batch['label']

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
train_features, test_features, train_labels, test_labels = train_test_split(features, y,  test_size=0.3,random_state=50)
#Import svm model
from sklearn import svm
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters,cv=10)
clf.fit(train_features,train_labels)
